## **Unzip the File**

In [ ]:
import zipfile

# Specify the name of the uploaded zip file
zip_file_name = '/content/drive/MyDrive/Capstone(Face, Speech, Recorder).zip'

# Specify the directory where you want to extract the contents
extract_dir = '/content/Models'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Optionally, you can print a message to confirm that the extraction was successful
print("Extraction complete.")

Extraction complete.


## **Libraries**

In [1]:
from IPython.display import display, Javascript, Image as IPythonImage
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from PIL import Image
import io
import tensorflow as tf
from tensorflow.keras.optimizers import Adamax
from io import BytesIO
from google.colab import output
from IPython.display import Javascript
import IPython.display as ipd
import wave
import pickle
from tensorflow.keras.models import Sequential, model_from_json
import librosa

## **Face Recognition**

In [2]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/face_recognition.h5', compile=False)
loaded_model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [3]:
image_path = '/content/Models/Face Recognition/Face photos/test/happy/im0.png'
image = Image.open(image_path)

# Preprocess the image
img = image.resize((224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

# Make predictions
predictions = loaded_model.predict(img_array)
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
score = tf.nn.softmax(predictions[0])
print(f"{class_labels[tf.argmax(score)]}")

1/1 [==============================] - 7s 7s/step
Happy


In [4]:
image_path = '/content/Models/Face Recognition/Face photos/test/fearful/im1002.png'
image = Image.open(image_path)

# Preprocess the image
img = image.resize((224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

# Make predictions
predictions = loaded_model.predict(img_array)
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
score = tf.nn.softmax(predictions[0])
print(f"{class_labels[tf.argmax(score)]}")

1/1 [==============================] - 0s 448ms/step
Fear


## **Testing with Live photo**

In [5]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# Use the function to capture the photo
filename = take_photo()

# Open the captured image and flip it
image = cv2.imread(filename)
flipped_image = cv2.flip(image, 1)  # Flip horizontally

# Save the flipped image
cv2.imwrite(filename, flipped_image)

# Display the flipped image using IPython.display.Image
display(IPythonImage(filename))

In [7]:
image_path = '/content/photo.jpg'
image = Image.open(image_path)

# Preprocess the image
img = image.resize((224, 224))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

# Make predictions
predictions = loaded_model.predict(img_array)
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
score = tf.nn.softmax(predictions[0])
print(f"{class_labels[tf.argmax(score)]}")

1/1 [==============================] - 11s 11s/step
Sad


## **Voice Recognition**

In [8]:
from tensorflow.keras.models import model_from_json
import librosa
import numpy as np
import pickle

# Load the Keras model from JSON file
def load_model(json_path, weights_path):
    with open(json_path, 'r') as json_file:
        loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(weights_path)
    print("Loaded model from disk")
    return loaded_model

loaded_model = load_model('/content/Models/Speech Recognition/Model/CNN_model.json',
                          '/content/Models/Speech Recognition/Model/CNN_model_weights.h5')

# Load scaler and encoder
def load_pickle(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

scaler2 = load_pickle('/content/drive/MyDrive/scaler2.pickle')
encoder2 = load_pickle('/content/drive/MyDrive/encoder2.pickle')

# Feature extraction functions
def zcr(data, frame_length, hop_length):
    zcr = librosa.feature.zero_crossing_rate(data, frame_length=frame_length, hop_length=hop_length)
    return np.squeeze(zcr)

def rmse(data, frame_length=2048, hop_length=512):
    rmse = librosa.feature.rms(y=data, frame_length=frame_length, hop_length=hop_length)
    return np.squeeze(rmse)

def mfcc(data, sr, frame_length=2048, hop_length=512, flatten: bool = True):
    mfccs = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=20, hop_length=hop_length, n_fft=frame_length)
    return np.squeeze(mfccs.T) if not flatten else np.ravel(mfccs.T)

def extract_features(data, sr=22050, frame_length=2048, hop_length=512):
    features = np.hstack((
        zcr(data, frame_length, hop_length),
        rmse(data, frame_length, hop_length),
        mfcc(data, sr, frame_length, hop_length)
    ))
    return features

def get_predict_feat(path):
    data, s_rate = librosa.load(path, duration=2.5, offset=0.6)
    features = extract_features(data)
    features = np.reshape(features, newshape=(1, -1))
    scaled_features = scaler2.transform(features)
    scaled_features = np.expand_dims(scaled_features, axis=2)
    return scaled_features

# Prediction function
def prediction(path1):
    res = get_predict_feat(path1)
    predictions = loaded_model.predict(res)
    y_pred = np.argmax(predictions, axis=1)
    emotions = {0:'Neutral', 1:'Calm', 2:'Happy', 3:'Sad', 4:'Angry', 5:'Fear', 6:'Disgust', 7:'Surprise'}
    print(emotions[y_pred[0]])

Loaded model from disk


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
prediction("/content/Models/Speech Recognition/Audio Data/Actor_03/03-01-04-02-01-02-03.wav")

1/1 [==============================] - 0s 346ms/step
Fear


In [10]:
prediction("/content/Models/Speech Recognition/Audio Data/Actor_07/03-01-06-02-01-01-07.wav")

1/1 [==============================] - 0s 114ms/step
Happy


In [11]:
prediction("/content/Models/Speech Recognition/Audio Data/Actor_03/03-01-01-01-01-01-03.wav")

1/1 [==============================] - 0s 110ms/step
Angry


## **Recorder**

In [15]:
from IPython.display import HTML, Audio
from google.colab import output
from base64 import b64decode
import io

# JavaScript to handle microphone recording and provide a "Record" and "Stop" button
def record_audio(filename='audio.wav', seconds=5):
    RECORD_AUDIO_HTML = f"""
    <script>
    function recordAudio(recorder, resolve) {{
        recorder.start();
        setTimeout(() => {{
            recorder.stop();
        }}, {seconds * 1000});
    }}

    function handleSuccess(stream, resolve) {{
        const options = {{ mimeType: 'audio/webm' }};
        const recordedChunks = [];
        const mediaRecorder = new MediaRecorder(stream, options);

        mediaRecorder.addEventListener('dataavailable', function(e) {{
            if (e.data.size > 0) {{
                recordedChunks.push(e.data);
            }}
        }});

        mediaRecorder.addEventListener('stop', function() {{
            const blob = new Blob(recordedChunks);
            const reader = new FileReader();
            reader.readAsDataURL(blob);
            reader.onloadend = function() {{
                const base64data = reader.result;
                resolve(base64data);
            }};
        }});

        recordAudio(mediaRecorder, resolve);
    }}

    const handleReject = (e) => {{
        console.log('navigator.MediaDevices.getUserMedia error: ', e);
    }}

    async function startRecording() {{
        const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
        new Promise((resolve) => handleSuccess(stream, resolve))
        .then((base64data) => {{
            google.colab.kernel.invokeFunction('notebook.SaveAudio', [base64data], {{}});
        }});
    }}

    startRecording();
    </script>
    """
    display(HTML(RECORD_AUDIO_HTML))
    output.register_callback('notebook.SaveAudio', save_audio)

def save_audio(base64_audio):
    header, data = base64_audio.split(',')
    binary_data = b64decode(data)
    with open('audio.wav', 'wb') as f:
        f.write(binary_data)
    print("Audio recording saved as 'audio.wav'")

In [21]:
# Call this function to start recording
record_audio(seconds=5)

Audio recording saved as 'audio.wav'


In [22]:
Audio('audio.wav')  # Playback the recorded audio

In [23]:
prediction('audio.wav')

<ipython-input-8-5570931356f9>:48: UserWarning: PySoundFile failed. Trying audioread instead.
  data, s_rate = librosa.load(path, duration=2.5, offset=0.6)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1/1 [==============================] - 0s 104ms/step
Neutral
